In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import sys
import time
import datetime
import argparse
import tensorboardX
import torch
import yaml
import shutil
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn
sys.path.append('../')

In [2]:
from pycode.dataset import RLBench_dataset, Softargmax_dataset, imageaug_full_transform, train_val_split, pose_aug
from pycode.config import _C as cfg
from pycode.model.Hourglass import stacked_hourglass_model
from pycode.loss.mp_loss import Train_Loss_sequence_hourglass
from pycode.misc import save_outputs, build_model_MP, build_dataset_MP, build_optimizer, str2bool, save_args, save_checkpoint, load_checkpoint

kornia requires version >= 3.6. your version 3.6


In [3]:
# parser
parser = argparse.ArgumentParser(description='parser for image generator')
parser.add_argument('--config_file', type=str, default='', metavar='FILE', help='path to config file')
parser.add_argument('--log_step', type=int, default=10, help='')
parser.add_argument('--save_step', type=int, default=100, help='')
parser.add_argument('--eval_step', type=int, default=100, help='')
parser.add_argument('--output_dirname', type=str, default='', help='')
parser.add_argument('--checkpoint_path', type=str, default=None, help='')
parser.add_argument('--log2wandb', type=str2bool, default=True)
args = parser.parse_args(args=['--config_file','../configs/RLBench_MP.yaml','--output_dirname','hoge', '--log2wandb','False'])
# args = parser.parse_args(args=['--checkpoint_path','output/2020-04-02_18:28:18.736004/model_log/checkpoint_epoch9_iter11'])

# get cfg data
if len(args.config_file) > 0:
    print('Loaded configration file {}'.format(args.config_file))
    cfg.merge_from_file(args.config_file)

# define output dirname
if len(args.output_dirname) == 0:
    dt_now = datetime.datetime.now()
    output_dirname = str(dt_now.date()) + '_' + str(dt_now.time())
else:
    output_dirname = args.output_dirname
    
cfg.BASIC.OUTPUT_DIR = os.path.join(cfg.BASIC.OUTPUT_DIR, cfg.DATASET.NAME, output_dirname)
cfg.freeze()

# define save model path
model_path = os.path.join(cfg.BASIC.OUTPUT_DIR, 'model_log')

# make output dir
os.makedirs(cfg.BASIC.OUTPUT_DIR, exist_ok=True)
os.makedirs(model_path, exist_ok=True)

# copy config file
if len(args.config_file) > 0:
    shutil.copy(args.config_file,cfg.BASIC.OUTPUT_DIR)

# save args
argsfile_path = os.path.join(cfg.BASIC.OUTPUT_DIR, "args.txt")
save_args(args,argsfile_path)

# set seed and cuda
torch.manual_seed(cfg.BASIC.SEED)
cuda = torch.cuda.is_available()
device = torch.device(cfg.BASIC.DEVICE)

if cuda:
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed(cfg.BASIC.SEED)

with open(args.config_file) as file:
    obj = yaml.safe_load(file)

if args.log2wandb:
    import wandb
    wandb.login()
    run = wandb.init(project='MotionPrediction-{}'.format(cfg.DATASET.NAME), entity='tendon', config=obj, save_code=True, name=args.output_dirname)

Loaded configration file ../configs/RLBench_MP.yaml


In [4]:
# set dataset
train_dataset = build_dataset_MP(cfg, save_dataset=False, mode='train')
val_dataset = build_dataset_MP(cfg, save_dataset=False, mode='val')

# set dataloader
train_dataloader = DataLoader(train_dataset, batch_size=cfg.BASIC.BATCH_SIZE, shuffle=True, num_workers=cfg.BASIC.WORKERS)
val_dataloader = DataLoader(val_dataset, batch_size=cfg.BASIC.BATCH_SIZE, shuffle=False, num_workers=cfg.BASIC.WORKERS)

# set model
model = build_model_MP(cfg)
# wandb.watch(model, log_freq=5000)
# model = torch.nn.DataParallel(model, device_ids = list(range(cfg.BASIC.NUM_GPU)))
# model = convert_model(model)
model = model.to(device)

# set loss
train_loss = Train_Loss_sequence_hourglass(cfg, device)
val_loss = Train_Loss_sequence_hourglass(cfg, device)

# set optimizer
optimizer = build_optimizer(cfg, model, 'mp')
scheduler = StepLR(optimizer, step_size=cfg.SCHEDULER.STEPLR.STEP_SIZE, gamma=cfg.SCHEDULER.STEPLR.GAMMA)

# load checkpoint
if args.checkpoint_path != None:
    checkpoint_path = os.path.join(args.checkpoint_path, 'mp.pth')
    
    if cfg.LOAD_MODEL == 'all':
        model, optimizer, start_epoch, start_iter, scheduler = load_checkpoint(model, checkpoint_path, optimizer=optimizer, scheduler=scheduler)
    elif cfg.LOAD_MODEL == 'model_only':
        model, _, _, _, _ = load_checkpoint(model, checkpoint_path)
        start_epoch, start_iter = 0, 1
else:
    start_epoch, start_iter = 0, 1

length of future is 1 frame
load json data
length of future is 1 frame
load json data
use hourglass


In [5]:
from ptflops import get_model_complexity_info
from ptflops.flops_counter import add_flops_counting_methods

model = build_model_MP(cfg)
model = model.to(device)
flops_model = add_flops_counting_methods(model)
flops_model.eval()
flops_model.start_flops_count(ost=sys.stdout, verbose=False, ignore_list=[])

use hourglass


In [6]:
for data in train_dataloader:
    break

In [7]:
_ = flops_model(data)

In [8]:
from ptflops.flops_counter import print_model_with_flops
from ptflops.flops_counter import flops_to_string, params_to_string

flops_count, params_count = flops_model.compute_average_flops_cost()
print_model_with_flops(flops_model, flops_count, params_count, ost=sys.stdout)
flops_model.stop_flops_count()

stacked_hourglass_model(
  27.729 M, 100.000% Params, 47.054 GMac, 100.000% MACs, 
  (initial_conv): ModuleList(
    2.238 M, 8.070% Params, 10.482 GMac, 22.277% MACs, 
    (0): ConvBlock(
      0.007 M, 0.025% Params, 0.034 GMac, 0.071% MACs, 
      (conv): Conv2d(0.007 M, 0.025% Params, 0.033 GMac, 0.070% MACs, 6, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): ReLU(0.0 M, 0.000% Params, 0.001 GMac, 0.001% MACs, )
    )
    (1): ResidualBlock(
      0.623 M, 2.248% Params, 2.92 GMac, 6.207% MACs, 
      (conv1): ConvBlock(
        0.148 M, 0.532% Params, 0.691 GMac, 1.470% MACs, 
        (conv): Conv2d(0.148 M, 0.532% Params, 0.691 GMac, 1.468% MACs, 128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act): ReLU(0.0 M, 0.000% Params, 0.001 GMac, 0.001% MACs, )
      )
      (conv2): ConvBlock(
        0.148 M, 0.532% Params, 0.691 GMac, 1.470% MACs, 
        (conv): Conv2d(0.148 M, 0.532% Params, 0.691 GMac, 1.468% MACs, 128, 128, kernel_size=(3

In [9]:
print(flops_to_string(flops_count))
print(params_to_string(params_count))

47.05 GMac
27.73 M
